In [2]:
import pandas as pd
import numpy as np
import requests
import os
import html
from LogUtil import create_logger
from lxml import etree
import feedparser
from SecEdgarDAO import SecEdgarDAO
from XBRL import lookup_cik
from MorningstarDAO import MorningstarDAO
import pandas as pd
import feedparser
import requests
import zipfile
import io
import datetime
import calendar

In [20]:
symbol = 'AAU'

url = 'http://financials.morningstar.com/cmpind/company-profile/component.action?component=OperationDetails&t=%s'
full_url = url % symbol

df = pd.read_html(requests.get(full_url).text, index_col=0)[0]
'%010d' % df.loc['CIK']

ValueError: No tables found

In [18]:
'%010d' % df.loc['CIK']

'0001672326'

In [7]:
df.loc['CIK']

1    1672326
Name: CIK, dtype: object

In [29]:
import feedparser
from requests.utils import quote


def lookup_cik_by_name(name):
    url = 'https://www.sec.gov/cgi-bin/browse-edgar?company=%s&owner=exclude&action=getcompany&output=atom'
    full_url = url % quote(name)
    #/feed/company-info/cik
    #/feed/entry/company-info/cik

    atom = feedparser.parse(full_url)
    if 'cik' in atom.feed:
        return [atom.feed.cik]
    
    return [entry['cik'] for entry in atom.entries]

#name = 'Yuma Energy, Inc.'
name = 'ImmunoCellular Therapeutics, Ltd.'
lookup_cik_by_name(name)

['0000822411']

False

In [20]:
feed.feed.entries

AttributeError: object has no attribute 'entries'

In [3]:
url = 'https://www.sec.gov/Archives/edgar/data/81318/000135448815003907/index.xml'

response = requests.get(url, stream=True, timeout=5)
doc = etree.fromstring(response.content)

In [12]:
for elem in doc.iter('name'):
    name = elem.text
    if name.endswith('.xsd'):
        print(name.replace('.xsd', '.xml'))

yuma-20150630.xml


In [5]:
for name in doc.xpath("/directory/name"):
    print(name)

<Element name at 0x7fc4211d2f48>


In [84]:
doc.find("dei:EntityCommonStockSharesOutstanding", doc.nsmap)

<Element {http://xbrl.sec.gov/dei/2014-01-31}EntityCommonStockSharesOutstanding at 0x7f0e1a2f35c8>

In [8]:
feed = feedparser.parse("https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000769397&type=10-%25&dateb=&owner=exclude&start=0&count=10&output=atom")

In [52]:
for entry in feed.entries:
    filing_href = entry['filing-href']
    print(filing_href)
    print(filing_href[0:filing_href.rfind('/')+1])

http://www.sec.gov/Archives/edgar/data/769397/000076939718000011/0000769397-18-000011-index.htm
http://www.sec.gov/Archives/edgar/data/769397/000076939718000011/
http://www.sec.gov/Archives/edgar/data/769397/000076939717000057/0000769397-17-000057-index.htm
http://www.sec.gov/Archives/edgar/data/769397/000076939717000057/
http://www.sec.gov/Archives/edgar/data/769397/000076939717000051/0000769397-17-000051-index.htm
http://www.sec.gov/Archives/edgar/data/769397/000076939717000051/
http://www.sec.gov/Archives/edgar/data/769397/000076939717000031/0000769397-17-000031-index.htm
http://www.sec.gov/Archives/edgar/data/769397/000076939717000031/
http://www.sec.gov/Archives/edgar/data/769397/000076939717000014/0000769397-17-000014-index.htm
http://www.sec.gov/Archives/edgar/data/769397/000076939717000014/
http://www.sec.gov/Archives/edgar/data/769397/000076939716000097/0000769397-16-000097-index.htm
http://www.sec.gov/Archives/edgar/data/769397/000076939716000097/
http://www.sec.gov/Archives/

In [91]:
class SecEdgarDAO(object):

    def __init__(self):
        self.url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=%s&type=10-%%25&dateb=&owner=exclude&start=0&count=10&output=atom"

    def _normalise_date(self, date):
        last_day_of_month = calendar.monthrange(date.year, date.month)[1]
        if last_day_of_month != date.day:
            if date.day <= 15:
                new_month = date.month - 1 if date.month > 1 else 12
                new_day = calendar.monthrange(date.year, new_month)[1]
                new_year = date.year if new_month != 12 else date.year - 1
                date = datetime.date(new_year, new_month, new_day)
            else:
                date = datetime.date(date.year, date.month, last_day_of_month)
        return date

    def _load_filings(self, cik, ignore_amended=False):
        full_url = self.url % (cik)
        feed = feedparser.parse(full_url)
        
        date_dict = {}
        for entry in feed.entries:
            filing_type = entry['filing-type']
            # skip amended
            if ignore_amended and filing_type not in ['10-Q', '10-K']:
                continue
            href = entry['filing-href']
            filing_date = entry['filing-date']
            folder = href[0:href.rfind('/')]
            summary = folder + '/FilingSummary.xml'
            
            doc = etree.fromstring(requests.get(summary, stream=True, timeout=5).content)
            xbrl_file = doc.xpath("/FilingSummary/MyReports/Report")[0].attrib['instance']
            xbrl_file = xbrl_file.replace('.htm', '_htm.xml')
            
            tree = etree.fromstring(requests.get(folder + '/' + xbrl_file, stream=True, timeout=5).content)
                        
            ns_xbrli = {'xbrli': 'http://www.xbrl.org/2003/instance'}
            
            shares_outstanding_node = tree.find("dei:EntityCommonStockSharesOutstanding", tree.nsmap)
            shares_outstanding_value = shares_outstanding_node.text
            context_ref_id = shares_outstanding_node.attrib['contextRef']
            instant = tree.find(
                "xbrli:context[@id = '%s']/xbrli:period/xbrli:instant" % context_ref_id, ns_xbrli)
            end_period = tree.find("dei:DocumentPeriodEndDate", tree.nsmap).text
            end_period_date = pd.to_datetime(end_period).date()
            norm_end_period_date = self._normalise_date(end_period_date)
            date_dict[norm_end_period_date] = {'xbrl_file': xbrl_file,
                                     'filing_type': filing_type,
                                     'filing_date': filing_date,
                                     'end_period': end_period,
                                     'shares_outstanding': shares_outstanding_value,
                                     'shares_outstanding_date': instant.text}

        return date_dict


    def get_filings_dates(self, cik, period_dates=None):
        date_dict = self._load_filings(cik, ignore_amended=True)
        if period_dates is None:
            period_dates = date_dict.keys()

        try:
            return [date_dict[x]['filing_date'] for x in period_dates]
        except KeyError as e:
            raise KeyError("Not all %s in %s" % (period_dates, date_dict.keys()))


    def get_shares_outstanding(self, cik):
        date_dict = self._load_filings(cik)
        return {date_dict[end_period]['shares_outstanding_date']: date_dict[end_period]['shares_outstanding'] for end_period in date_dict.keys()}


edgar = SecEdgarDAO()
#cik = '0001341439' #AAPL
#cik = '0001593195' #TRNC
cik = '0000769397' #ADSK
edgar._load_filings(cik, ignore_amended=True)


{datetime.date(2015, 10, 31): {'end_period': '2015-10-31',
  'filing_date': '2015-12-09',
  'filing_type': '10-Q',
  'shares_outstanding': '225323161',
  'shares_outstanding_date': '2015-11-30',
  'xbrl_file': 'adsk-20151031.xml'},
 datetime.date(2016, 1, 31): {'end_period': '2016-01-31',
  'filing_date': '2016-03-23',
  'filing_type': '10-K',
  'shares_outstanding': '224493192',
  'shares_outstanding_date': '2016-03-09',
  'xbrl_file': 'adsk-20160131.xml'},
 datetime.date(2016, 4, 30): {'end_period': '2016-04-30',
  'filing_date': '2016-06-06',
  'filing_type': '10-Q',
  'shares_outstanding': '224656057',
  'shares_outstanding_date': '2016-05-31',
  'xbrl_file': 'adsk-20160430.xml'},
 datetime.date(2016, 7, 31): {'end_period': '2016-07-31',
  'filing_date': '2016-08-30',
  'filing_type': '10-Q',
  'shares_outstanding': '221892961',
  'shares_outstanding_date': '2016-08-23',
  'xbrl_file': 'adsk-20160731.xml'},
 datetime.date(2016, 10, 31): {'end_period': '2016-10-31',
  'filing_date':